In [22]:
import pandas as pd
from pandas.api.types import CategoricalDtype

In [2]:
df = pd.read_csv("data/yellow_tripdata_combined_cleaned.csv")

In [5]:
df.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_at,dropoff_at,duration
0,0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,...,0.5,3.65,0.0,0.3,21.95,2.5,0.0,2022-01-01 00:35:40,2022-01-01 00:53:29,1069.0
1,1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,...,0.5,4.00,0.0,0.3,13.30,0.0,0.0,2022-01-01 00:33:43,2022-01-01 00:42:07,504.0
2,2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,...,0.5,1.76,0.0,0.3,10.56,0.0,0.0,2022-01-01 00:53:21,2022-01-01 01:02:19,538.0
3,3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,...,0.5,0.00,0.0,0.3,11.80,2.5,0.0,2022-01-01 00:25:21,2022-01-01 00:35:23,602.0
4,4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,...,0.5,3.00,0.0,0.3,30.30,2.5,0.0,2022-01-01 00:36:48,2022-01-01 01:14:20,2252.0


In [3]:
df["pickup_at"] = pd.to_datetime(df["pickup_at"])
df["dropoff_at"] = pd.to_datetime(df["dropoff_at"])
df["pickup_at_hour"] = df["pickup_at"].dt.round("H")

In [4]:
df_weather = pd.read_csv("data/weather_daily.csv", index_col=False)
df_weather["time"] = pd.to_datetime(df_weather["time"])

In [5]:
df_weather.head()

,time,precipitation (mm)
0,2022-01-01 00:00:00,0.0
1,2022-01-01 01:00:00,0.0
2,2022-01-01 02:00:00,0.0
3,2022-01-01 03:00:00,0.7
4,2022-01-01 04:00:00,0.0


In [5]:
# Merge trips data (df) with weather data for that particular hour
# (df_weather) on the "time" column.
#
#small["pickup_at"].dt.round("H")
merged = pd.merge(df, df_weather, left_on="pickup_at_hour", right_on="time")

In [6]:
# Make new column "weather" that is "rainy" if precipitation > 0, else "sunny".

merged["weather"] = "sunny"
merged["weather"][merged["precipitation (mm)"] > 0] = "rainy"

/var/folders/vs/ylfhyw117z1g5xvn55z8jr3c0000gp/T/ipykernel_81571/4246521071.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["weather"][merged["precipitation (mm)"] > 0] = "rainy"


In [7]:
# day/night, sunny/rainy, spring/summer/fall/winter, area (manhattan/all)
merged["time"] = "day"
merged["time"][20 < merged["pickup_at"].dt.hour] = "night"
merged["time"][merged["pickup_at"].dt.hour < 6] = "night"

/var/folders/vs/ylfhyw117z1g5xvn55z8jr3c0000gp/T/ipykernel_81571/2682998250.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["time"][20 < merged["pickup_at"].dt.hour] = "night"
/var/folders/vs/ylfhyw117z1g5xvn55z8jr3c0000gp/T/ipykernel_81571/2682998250.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["time"][merged["pickup_at"].dt.hour < 6] = "night"


In [8]:
merged["pickup_area"] = "not manhattan"
merged["dropoff_area"] = "not manhattan"
manhattan_location_ids = [103, 104, 105, 12, 88, 87, 209, 261, 13, 45, 231, 232, 148, 144, 211, 125, 4, 79, 114, 113, 249, 158, 224, 107, 234, 90, 68, 246, 186, 164, 137, 170, 233, 100, 229, 162, 161, 230, 48, 50, 202, 140, 141, 237, 142, 143, 43, 262, 263, 236, 239, 238, 75, 151, 24, 74, 41, 166, 194, 42, 152, 116, 244, 120, 243, 127, 128, 153]
merged["pickup_area"][merged["PULocationID"].isin(manhattan_location_ids)] = "manhattan"
merged["dropoff_area"][merged["DOLocationID"].isin(manhattan_location_ids)] = "manhattan"

/var/folders/vs/ylfhyw117z1g5xvn55z8jr3c0000gp/T/ipykernel_81571/3562136131.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["pickup_area"][merged["PULocationID"].isin(manhattan_location_ids)] = "manhattan"
/var/folders/vs/ylfhyw117z1g5xvn55z8jr3c0000gp/T/ipykernel_81571/3562136131.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["dropoff_area"][merged["DOLocationID"].isin(manhattan_location_ids)] = "manhattan"


In [9]:
merged["season"] = "winter"
merged["season"][(merged["pickup_at"].dt.month > 2) & (merged["pickup_at"].dt.month < 6)] = "spring"
merged["season"][(merged["pickup_at"].dt.month > 5) & (merged["pickup_at"].dt.month < 9)] = "summer"
merged["season"][(merged["pickup_at"].dt.month > 8) & (merged["pickup_at"].dt.month < 12)] = "fall"

/var/folders/vs/ylfhyw117z1g5xvn55z8jr3c0000gp/T/ipykernel_81571/3454911036.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["season"][(merged["pickup_at"].dt.month > 2) & (merged["pickup_at"].dt.month < 6)] = "spring"
/var/folders/vs/ylfhyw117z1g5xvn55z8jr3c0000gp/T/ipykernel_81571/3454911036.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["season"][(merged["pickup_at"].dt.month > 5) & (merged["pickup_at"].dt.month < 9)] = "summer"
/var/folders/vs/ylfhyw117z1g5xvn55z8jr3c0000gp/T/ipykernel_81571/3454911036.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [64]:
dayname_type = CategoricalDtype(categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"], ordered=True)
by_dayname = merged.groupby([
    merged["pickup_at"].dt.day_name().astype(dayname_type),
    merged["weather"], 
    merged["season"], 
    merged["time"], 
    merged["pickup_area"]
])

In [71]:
by_dayname["trip_distance"].mean().to_csv("output/avg-trip-distance_by-day.csv")
by_dayname["duration"].mean().to_csv("output/avg-duration_by-day.csv")
by_dayname["tip_amount"].mean().to_csv("output/avg-tip-amount_by-day.csv")
by_dayname["duration"].count().to_csv("output/number-of-trips_by-day.csv")

In [16]:
merged["duration"].groupby([merged["PULocationID"], merged["weather"], merged["season"], merged["time"], merged["pickup_area"]]).mean().to_csv("output/avg-duration_by-pickup-zone.csv")
merged["duration"].groupby([merged["PULocationID"], merged["weather"], merged["season"], merged["time"], merged["pickup_area"]]).count().to_csv("output/number-of-trips_by-pickup-zone.csv")
merged["trip_distance"].groupby([merged["PULocationID"], merged["weather"], merged["season"], merged["time"], merged["pickup_area"]]).mean().to_csv("output/avg-trip-distance_by-pickup-zone.csv")
merged["tip_amount"].groupby([merged["PULocationID"], merged["weather"], merged["season"], merged["time"], merged["pickup_area"]]).mean().to_csv("output/avg-tip-amount_by-pickup-zone.csv")